The code from `march1.ipynb` is modified in one overarching respect: The input csv-file contains both descrete and continous features (X columns). The indeces of the descrete columns (as well as name of the file) are hardcoded below but they are of course meant to become user-provided arguments. The output is a dictionary, where the keys are products of dimensions of contingency_matrices encountered in the computation, and values are again dictionaries similar to the one used before as a whole output i.e. maps from column-indeces to maximal information gains (and corresponding tuples).

In order to produce an input file, every `n`th column of `madelon.csv` will be discretized. Details of the way the partially-discretized csv is generated is fairly obvious form the code below. The resulting file is saved as a new file `madelon_n.csv` and the orginal `madelon.csv` is not used anymore.

---
Reminder:
* Swap error handling in `matrix.h` for assertions.

---
The `cpp` code

In [3]:
%%writefile wrap_discretize.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>
*/

#include <algorithm>
#include <cstdint>
#include <cstring>
#include <random>
#include <vector>
#include <pybind11/pybind11.h>
//#include <pybind11/stl.h>
#include <pybind11/numpy.h>

namespace py = pybind11;
using namespace pybind11::literals;

void discretize(
    uint32_t seed,
    uint32_t discretization_index,
    uint32_t feature_id,
    std::size_t divisions,
    std::size_t object_count,
    py::array_t<double> py_in_data,
    py::array_t<double> py_sorted_in_data,
    py::array_t<uint8_t> py_out_data,
    double range_
) {
    
    // the python part
    py::buffer_info py_in_data_buf = py_in_data.request();
    auto *in_data = static_cast<const double *>(py_in_data_buf.ptr);    
    
    py::buffer_info py_sorted_in_data_buf = py_sorted_in_data.request();
    auto *sorted_in_data = static_cast<const double *>(py_sorted_in_data_buf.ptr);
    
    py::buffer_info py_out_data_buf = py_out_data.request();
    auto *out_data = static_cast<uint8_t *>(py_out_data_buf.ptr);
    
    // end of the python part
    
    double* thresholds = new double[divisions];

    // brackets to limit scope
    {
        double sum = 0.0f;
        // brackets to limit scope
        {
            std::mt19937 seed_random_generator0(seed);
            std::mt19937 seed_random_generator1(seed_random_generator0() ^ discretization_index);
            std::mt19937 random_generator(seed_random_generator1() ^ feature_id);

            // E(X) = (a + b) / 2 = (1 - range + 1 + range) / 2 = 1
            std::uniform_real_distribution<double> uniform_range(1.0f - range_, 1.0f + range_);

            for (std::size_t d = 0; d < divisions; ++d) {
                thresholds[d] = uniform_range(random_generator);
                sum += thresholds[d];
            }

            sum += uniform_range(random_generator);
        }

        std::size_t done = 0;
        const double length_step = static_cast<double>(object_count) / sum;

        // thresholds are converted from an arbitrary space into real values (via indices)
        // d - iterates over divisions (of a variable in a discretization)
        for (std::size_t d = 0; d < divisions; ++d) {
            done += std::lround(thresholds[d] * length_step);

            // Note: Check when will this happen, maybe could be skipped
            if (done >= object_count) {
                done = object_count - 1;
            }

            thresholds[d] = sorted_in_data[done];
        }
    }

    // o - iterates over objects
    for (std::size_t o = 0; o < object_count; ++o) {
        out_data[o] = 0;

        // out_data[o] (starting with 0) is incremented every time in_data[o] is above a threshold
        // divisions is a small number (<=15), no reason to use binsearch, hence linear
        // d - iterates over divisions (per object o)
        for (std::size_t d = 0; d < divisions; ++d) {
            out_data[o] += in_data[o] > thresholds[d];
        }
    }

    delete[] thresholds;
}


PYBIND11_MODULE(wrap_discretize, module) {
    module.doc() = "Wrapper of https://bitbucket.org/mdfs/mdfs/src/master/src/cpu/discretize.cpp";
    module.def("discretize", &discretize,
               "In place discretization.",
               "seed"_a,
               "discretization_index"_a,
               "feature_id"_a,
               "divisions"_a,
               "object_count"_a,
               "py_in_data"_a,
               "py_sorted_in_data"_a,
               "py_out_data"_a,
               "range_"_a
              );
}

Overwriting wrap_discretize.cpp


In [4]:
%%writefile fast.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>
*/

#include <math.h>
#include <tuple>
#include <vector>
#include <pybind11/pybind11.h>
//#include <pybind11/stl.h>
#include <pybind11/numpy.h>
#include <omp.h>

#include "matrix_class.h"

namespace py = pybind11;

std::tuple<double, double, double> work_3a(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    Matrix<int> contingency_m(kC_Xdim, kC_Xdim, kC_Xdim, kC_ydim);
    Matrix<double> contingency_m_y(kC_Xdim, kC_Xdim, kC_Xdim);
    
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        ++contingency_m(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]);
    }

    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            for (int C_yidx = 0; C_yidx < kC_ydim; ++C_yidx) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_j, C_Xidx_k, C_yidx);
                    count_X0 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_k, C_Xidx_i, C_Xidx_j, C_yidx);
                    count_X1 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_k, C_Xidx_j, C_yidx);
                    count_X2 += count_ijk;
                    contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k) += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; ++C_Xidx_k) {
                double count_y_ijk = contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
                neg_H -= count_y_ijk * log2(count_y_ijk);
                count_X0_y += contingency_m_y(C_Xidx_k, C_Xidx_i, C_Xidx_j);
                count_X1_y += contingency_m_y(C_Xidx_i, C_Xidx_k, C_Xidx_j);
                count_X2_y += contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
    
    return std::make_tuple(neg_H - neg_H_X0,
                           neg_H - neg_H_X1,
                           neg_H - neg_H_X2
                          );    
}

std::tuple<double, double, double> work_3b(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    Matrix<int> contingency_m(kC_Xdim, kC_Xdim, kC_Xdim, kC_ydim);
    Matrix<double> contingency_m_y(kC_Xdim, kC_Xdim, kC_Xdim);
    
    Matrix<omp_lock_t> contingency_m_lock(kC_Xdim, kC_Xdim, kC_Xdim, kC_ydim);
                                                                                                
    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim; ++C_Xidx_0) {
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim; ++C_Xidx_1) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim; ++C_Xidx_2) {
                for (int C_yidx = 0; C_yidx < kC_ydim; ++C_yidx) {
                    omp_init_lock(&contingency_m_lock(C_Xidx_0, C_Xidx_1, C_Xidx_2, C_yidx));
                }
            }
        }
    }


    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for                                           
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        omp_set_lock(&contingency_m_lock(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]));
        ++contingency_m(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]);
        omp_unset_lock(&contingency_m_lock(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]));
    }
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            for (int C_yidx = 0; C_yidx < kC_ydim; ++C_yidx) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_j, C_Xidx_k, C_yidx);
                    count_X0 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_k, C_Xidx_i, C_Xidx_j, C_yidx);
                    count_X1 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_k, C_Xidx_j, C_yidx);
                    count_X2 += count_ijk;
                    contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k) += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; ++C_Xidx_k) {
                double count_y_ijk = contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
                neg_H -= count_y_ijk * log2(count_y_ijk);
                count_X0_y += contingency_m_y(C_Xidx_k, C_Xidx_i, C_Xidx_j);
                count_X1_y += contingency_m_y(C_Xidx_i, C_Xidx_k, C_Xidx_j);
                count_X2_y += contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
    
    return std::make_tuple(neg_H - neg_H_X0,
                           neg_H - neg_H_X1,
                           neg_H - neg_H_X2
                          );    
}

std::tuple<double, double, double> work_3c(const int kData_dim,
                                           const int kDivisions,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    Matrix<int> contingency_m(kC_Xdim, kC_Xdim, kC_Xdim, kC_ydim);
    Matrix<double> contingency_m_y(kC_Xdim, kC_Xdim, kC_Xdim);                                                                                                


    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for                                         
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        #pragma omp atomic
            ++contingency_m(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]);
    }
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            for (int C_yidx = 0; C_yidx < kC_ydim; ++C_yidx) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_j, C_Xidx_k, C_yidx);
                    count_X0 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_k, C_Xidx_i, C_Xidx_j, C_yidx);
                    count_X1 += pseudo_counts[C_yidx] + contingency_m(C_Xidx_i, C_Xidx_k, C_Xidx_j, C_yidx);
                    count_X2 += count_ijk;
                    contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k) += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; ++C_Xidx_i) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; ++C_Xidx_j) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; ++C_Xidx_k) {
                double count_y_ijk = contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
                neg_H -= count_y_ijk * log2(count_y_ijk);
                count_X0_y += contingency_m_y(C_Xidx_k, C_Xidx_i, C_Xidx_j);
                count_X1_y += contingency_m_y(C_Xidx_i, C_Xidx_k, C_Xidx_j);
                count_X2_y += contingency_m_y(C_Xidx_i, C_Xidx_j, C_Xidx_k);
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
    
    return std::make_tuple(neg_H - neg_H_X0,
                           neg_H - neg_H_X1,
                           neg_H - neg_H_X2
                          );    
}
    

std::tuple<double, double, double> work_3_old(const int kData_dim,
                                              const int kDivisions,
                                              py::array_t<int> &py_X0,
                                              py::array_t<int> &py_X1,
                                              py::array_t<int> &py_X2,
                                              const int kN_classes,
                                              py::array_t<double> &py_pseudo_counts,
                                              py::array_t<int> &py_y) {
    
    const int kC_Xdim = kDivisions + 1;
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = static_cast<int *>(py_y_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = static_cast<double *>(py_pseudo_counts_buf.ptr);
    
    int contingency_m[kC_Xdim][kC_Xdim][kC_Xdim][kC_ydim] = {};
    double contingency_m_y[kC_Xdim][kC_Xdim][kC_Xdim] = {};

    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        contingency_m[X0[data_idx]][X1[data_idx]][X2[data_idx]][y[data_idx]]++;
    }

    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    
    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                double count_X0 = 0;
                double count_X1 = 0;
                double count_X2 = 0;
                for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                    double count_ijk = pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_j][C_Xidx_k][C_yidx];
                    count_X0 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_k][C_Xidx_i][C_Xidx_j][C_yidx];
                    count_X1 += pseudo_counts[C_yidx] + contingency_m[C_Xidx_i][C_Xidx_k][C_Xidx_j][C_yidx];
                    count_X2 += count_ijk;
                    contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] += count_ijk;
                    neg_H += count_ijk * log2(count_ijk);
                }
                neg_H_X0 += count_X0 * log2(count_X0);
                neg_H_X1 += count_X1 * log2(count_X1);
                neg_H_X2 += count_X2 * log2(count_X2);
            }
        }
    }

    #pragma omp for reduction (+: neg_H, neg_H_X0, neg_H_X1, neg_H_X2)
    for (int C_Xidx_i = 0; C_Xidx_i < kC_Xdim; C_Xidx_i++) {
        for (int C_Xidx_j = 0; C_Xidx_j < kC_Xdim; C_Xidx_j++) {
            double count_X0_y = 0;
            double count_X1_y = 0;
            double count_X2_y = 0;
            for (int C_Xidx_k = 0; C_Xidx_k < kC_Xdim; C_Xidx_k++) {
                neg_H -= contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k] * log2(contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k]);
                count_X0_y += contingency_m_y[C_Xidx_k][C_Xidx_i][C_Xidx_j];
                count_X1_y += contingency_m_y[C_Xidx_i][C_Xidx_k][C_Xidx_j];
                count_X2_y += contingency_m_y[C_Xidx_i][C_Xidx_j][C_Xidx_k];
            }
            neg_H_X0 -= count_X0_y * log2(count_X0_y);
            neg_H_X1 -= count_X1_y * log2(count_X1_y);
            neg_H_X2 -= count_X2_y * log2(count_X2_y);
        }
    }
    
    }
        
    return std::make_tuple(neg_H - neg_H_X0,
                           neg_H - neg_H_X1,
                           neg_H - neg_H_X2
                          );
}
       

PYBIND11_MODULE(fast, module) {
    module.def("work_3a", &work_3a);
    module.def("work_3b", &work_3b);
    module.def("work_3c", &work_3c);
    module.def("work_3_old", &work_3_old);
}

Overwriting fast.cpp


Compilation

In [5]:
import cppimport
#cppimport.set_quiet(False)

fast = cppimport.imp("fast")
cppimport.imp("wrap_discretize")

<module 'wrap_discretize' from '/home/olszewskip/Desktop/git-repos/MDFS_playground/python/scheduler/ver0/wrap_discretize.cpython-35m-x86_64-linux-gnu.so'>

---
Make the partially-discretized csv file. Discretize every `n`th column.

In [6]:
import wrap_discretize
import numpy as np
from pandas import read_csv
import pandas as pd

data_df = read_csv("madelon.csv", header=None)
data_df_2 = data_df.copy()
in_data = np.ascontiguousarray(np.array(data_df.values.T, dtype='float64')[:-2])
out_data = np.zeros_like(in_data, dtype='uint8')

In [8]:
n = 5

for col_idx in range(0, in_data.shape[0] - 2):
    if col_idx % n == 0:
        wrap_discretize.discretize(
            seed = 123,
            discretization_index = 0,
            feature_id = col_idx,
            divisions = 1 + col_idx // n % n,
            object_count = in_data.shape[1],
            py_in_data = in_data[col_idx],
            py_sorted_in_data = np.sort(in_data[col_idx]),
            py_out_data = out_data[col_idx],
            range_ = 0
        )
        data_df_2[col_idx] = out_data[col_idx]

data_df_3 = pd.DataFrame(data_df_2.values.astype('float'))
data_df_3.to_csv('madelon_{}.csv'.format(n), header=False, index=False)

---
Make a data sample: pick `m` last X-columns in the `madelon_5`

In [9]:
m = 50

from pandas import read_csv

madelon = read_csv("madelon_5.csv", header=None)
madelon.iloc[:,-(m + 2):].to_csv('madelon_5_tiny.csv', header=False, index=False)
madelon_5_tiny = read_csv("madelon_5_tiny.csv", dtype = 'float64', header=None)
madelon_5_tiny.shape

discrete_cols = list(range(0, m, 5))
discrete_cols

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

---
Run the thing synchronously (make it a standalone script for profiling)

In [10]:
%%writefile march2_n1.py

import csv
import numpy as np
from scipy.stats import rankdata
from pandas import read_csv, DataFrame
from itertools import product, chain, starmap, combinations, combinations_with_replacement
from time import time
import pickle

import wrap_discretize
import fast

time0 = time()

k = 3
divisions = 7
range_ = 0.00
seed = 123
wrap_discretize_switch = True

# 1. Read in the data

file = "madelon_5_tiny.csv"
discrete_cols = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.ascontiguousarray(np.array(input_, dtype='float64').T[:-1])
data = np.zeros_like(input_, dtype='uint8')
bucket_counts = np.empty(input_.shape[0] - 1, dtype='int')

# Discretize the data

if wrap_discretize_switch:

    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            wrap_discretize.discretize(
                seed = 123,
                discretization_index = 0,
                feature_id = col_idx,  # ?
                divisions = divisions,
                object_count = input_.shape[1],
                py_in_data = input_[col_idx],
                py_sorted_in_data = np.sort(input_[col_idx]),
                py_out_data = data[col_idx],
                range_ = range_
            )
            bucket_counts[col_idx] = divisions + 1
else:
    
    def discretize(seq, divisions=divisions, range_=range_, seed=seed):
        '''
        >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
        where
        ranks = [2., 3., 1., 4.5, 6., 4.5]
        tresholds = [1.5,  3.,  4.5]
        '''
        np.random.seed(seed)
        ranks = rankdata(seq, method='average') # method='ordinal'/'average' ?

        random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
        tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)

        discrete_seq = np.zeros(len(seq), dtype='uint8')
        for treshold in tresholds:
            discrete_seq[ranks > treshold] += 1
        return discrete_seq

    # discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])
    
    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            data[col_idx] = discretize(input_[col_idx])
            bucket_counts[col_idx] = divisions + 1


data[-1] = input_[-1:].astype('uint8')

final_results = {dof: {} for dof in set(map(np.prod, tuple(combinations(bucket_counts, r=k))))}

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape

# 3. More function definitions

def tuple_generator(k=k, dim0=dim0):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,1}, {0,2}, ..., {0, dim0-1}, {1,2}, ..., {1,dim0-1}, ..., {dim0-2, dim0-1}
    Go with combinations_with_replacement() to include diagonal tuples like {0, 0}
    '''        
    return combinations(range(dim0), k)    

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(indeces):
    '''
    indeces -> tuple
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to the indeces
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty([divisions + 1] * k + [len(labels)], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[list(indeces) + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    results = []
    for i in range(len(indeces)):
        result = neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i))
        results.append(result)
    return tuple(results)


def record(tuple_, IGs, records):
    for column, IG in zip(tuple_, IGs):
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)


for tuple_ in tuple_generator():
    #IGs = slow_work(tuple_)
    IGs = fast.work_3a(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3b(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3c(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    #IGs = fast.work_3_old(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
    
    dof = np.prod(tuple(bucket_counts[col_idx] for col_idx in tuple_), dtype = 'int')
    record(tuple_, IGs, final_results[dof])

# result
print("Finished in", time() - time0, "sec.")

with open("march2_n1_results.pkl", "wb") as file:
    pickle.dump(final_results, file)

Overwriting march2_n1.py


In [11]:
%run march2_n1.py

Finished in 2.0947937965393066 sec.


In [12]:
import pandas as pd
import pickle
with open("march2_n1_results.pkl", "rb") as file:
    final_results_n1 = pickle.load(file)

for dof, dict_ in final_results_n1.items():
    
    df = pd.DataFrame(dict_).T.rename(columns={0: 'IG_max', 1: 'tuple'})
    df.sort_values('IG_max', ascending=False, inplace=True)

    print("product of X's bucket-counts = {}".format(dof))
    display(df)
    print("---")

product of X's bucket-counts = 128


,IG_max,tuple
1,273.836,"(1, 25, 43)"
22,268.547,"(22, 25, 43)"
25,239.06,"(1, 25, 43)"
43,221.95,"(1, 25, 43)"
3,205.555,"(3, 22, 25)"
14,124.049,"(6, 14, 25)"
6,121.529,"(6, 14, 25)"
42,118.579,"(0, 12, 42)"
12,118.251,"(0, 12, 42)"
21,117.97,"(21, 25, 32)"


---
product of X's bucket-counts = 512


,IG_max,tuple
43,469.58,"(1, 22, 43)"
3,457.692,"(1, 3, 22)"
22,412.102,"(3, 22, 41)"
1,407.952,"(1, 32, 43)"
9,392.656,"(7, 9, 14)"
18,390.184,"(18, 37, 47)"
7,383.371,"(7, 9, 14)"
14,379.428,"(7, 9, 14)"
38,373.26,"(6, 12, 38)"
48,372.887,"(12, 17, 48)"


---
product of X's bucket-counts = 64


...

---
product of X's bucket-counts = 192


,IG_max,tuple
43,284.9,"(1, 5, 43)"
22,278.071,"(5, 22, 43)"
3,265.697,"(1, 3, 5)"
1,257.315,"(1, 5, 43)"
5,180.757,"(5, 22, 43)"
21,166.651,"(5, 21, 27)"
26,166.587,"(5, 13, 26)"
28,165.338,"(17, 28, 30)"
17,163.5,"(17, 28, 30)"
41,162.253,"(22, 30, 41)"


---
product of X's bucket-counts = 72


,IG_max,tuple
43,103.065,"(5, 30, 43)"
22,101.528,"(5, 22, 30)"
25,96.6994,"(20, 25, 45)"
3,88.8426,"(3, 5, 30)"
5,68.595,"(5, 22, 30)"
21,61.0716,"(5, 21, 30)"
9,60.6375,"(5, 9, 30)"
38,59.7251,"(5, 30, 38)"
41,56.4259,"(5, 30, 41)"
46,55.5272,"(5, 30, 46)"


---


In [13]:
%%bash
python -m cProfile -o march2_n1.prof march1_n1.py

Finished in 1.4661047458648682 sec.


In [14]:
import pstats
p = pstats.Stats('march2_n1.prof')
#p.sort_stats('time').print_stats(15)
p.strip_dirs().sort_stats('time').print_stats(10)

Sat Mar 23 10:22:38 2019    march2_n1.prof

         498457 function calls (488264 primitive calls) in 2.063 seconds

   Ordered by: internal time
   List reduced from 3382 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    19600    1.326    0.000    1.326    0.000 {built-in method fast.work_3a}
        1    0.093    0.093    2.065    2.065 march1_n1.py:1(<module>)
      539    0.065    0.000    0.065    0.000 {built-in method marshal.loads}
    822/1    0.047    0.000    2.065    2.065 {built-in method builtins.exec}
   127/92    0.041    0.000    0.084    0.001 {built-in method _imp.create_dynamic}
    19600    0.037    0.000    0.037    0.000 march1_n1.py:122(record)
1574/1559    0.031    0.000    0.082    0.000 {built-in method builtins.__build_class__}
      297    0.022    0.000    0.054    0.000 doccer.py:12(docformat)
       48    0.014    0.000    0.014    0.000 config.py:414(register_option)
      870    0.014    0.000  

---
Make a script that uses mpi4py and run with mpi from bash

In [15]:
%%writefile march2_mpi.py
import csv
import numpy as np
from scipy.stats import rankdata
from itertools import product, chain, starmap, combinations, combinations_with_replacement
import pickle

import wrap_discretize
import fast

from mpi4py import MPI
comm = MPI.COMM_WORLD
comm.Barrier()
time0 = MPI.Wtime()
size = comm.Get_size()
rank = comm.Get_rank()

k = 3
window = 5
divisions = 7
range_ = 0.0
seed = 123
    
wrap_discretize_switch = True

# 1. Read in the data

file = "madelon_5_tiny.csv"
discrete_cols = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.ascontiguousarray(np.array(input_, dtype='float64').T[:-1])
data = np.zeros_like(input_, dtype='uint8')
bucket_counts = np.empty(input_.shape[0] - 1, dtype='int')

# Discretize the data

if wrap_discretize_switch:

    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            wrap_discretize.discretize(
                seed = 123,
                discretization_index = 0,
                feature_id = col_idx,  # ?
                divisions = divisions,
                object_count = input_.shape[1],
                py_in_data = input_[col_idx],
                py_sorted_in_data = np.sort(input_[col_idx]),
                py_out_data = data[col_idx],
                range_ = range_
            )
            bucket_counts[col_idx] = divisions + 1
else:
    
    def discretize(seq, divisions=divisions, range_=range_, seed=seed):
        '''
        >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
        where
        ranks = [2., 3., 1., 4.5, 6., 4.5]
        tresholds = [1.5,  3.,  4.5]
        '''
        np.random.seed(seed)
        ranks = rankdata(seq, method='average') # method='ordinal'/'average' ?

        random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
        tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)

        discrete_seq = np.zeros(len(seq), dtype='uint8')
        for treshold in tresholds:
            discrete_seq[ranks > treshold] += 1
        return discrete_seq

    #discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])
    
    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            data[col_idx] = discretize(input_[col_idx])
            bucket_counts[col_idx] = divisions + 1

    
data[-1] = input_[-1:].astype('uint8')

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape
M = (dim0 - 1) // window + 1
border_cols = range( (M-1) * window, dim0)

if rank == 0:
    print(rank, "dim0 =", dim0, "; dim1 =", dim1)

# 3. More function definitions

def tile_generator(k=k, M=M):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,0}, {0,1}, ..., {0, M-1}, {1,1}, ..., {1,M-1}, ..., {M-1, M-1}
    Go with combinations(range(M), k) to exclude diagonal tuples
    '''        
    return combinations_with_replacement(range(M), k)    

def tuple_generator(tile, window=window, border_cols=border_cols):
    '''
    Map tile into sequence of k-tuples, i.e. fundamental-tiles,
    i.e. elements of the cartesian product of the data-columns
    '''
    index_counts = {index: tile.count(index) for index in tile}
    index_to_cols = lambda index: range(index * window, (index + 1) * window) if index != (M - 1) else border_cols 
    cols_tile = (combinations(index_to_cols(index), count) for (index, count) in index_counts.items())
    return (list(chain.from_iterable(col_indeces)) for col_indeces in product(*cols_tile))

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(tuple_, divisions=divisions, n_classes=n_classes, pseudo_counts=pseudo_counts):
    '''
    tuple_ -> list # dammit...
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to column-indeces in the tuple_
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty([divisions + 1] * k + [n_classes], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[tuple_ + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    IGs = tuple(neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i)) for i in range(len(tuple_)))
    return IGs


def record_tuple(tuple_, IGs, dof, records):
    # records -> dict of dicts
    for column, IG in zip(tuple_, IGs):
        if dof not in records:
            records[dof] = {column: (IG, tuple_)}
        elif column not in records[dof] or IG > records[dof][column][0]:
            records[dof][column] = (IG, tuple_)
            
def record_tile(tile_results, records):
    # tile_results, records -> dicts of dicts
    for dof, dict_ in tile_results.items():
        for column, (IG, tuple_) in dict_.items():
            if column not in records[dof] or IG > records[dof][column][0]:
                records[dof][column] = (IG, tuple_)

# 4 Work loop

if rank == 0:
    final_results = {dof: {} for dof in set(map(np.prod, tuple(combinations(bucket_counts, r=k))))}
    current_assignements = {rank: (0, None) for rank in range(1,size)}
    
    print(rank, "entering the for loop")
    status = MPI.Status()
    for tile in tile_generator():
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(tile, dest=status.source)
        job_count = current_assignements[status.source][0]
        current_assignements[status.source] = (job_count + 1, tile)
        
        #print(rank, "currently:", current_assignements)
    
    print(rank, "Work queue is empty")
    for _ in range(size - 1):
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(None, dest=status.source)

    # Save the results to a file
    with open("march2_mpi_results.pkl", "wb") as file:
        pickle.dump(final_results, file)

    print(rank, "says goodbye")
    print(rank, "Elapsed:", MPI.Wtime() - time0, "sec")
    
else:
    comm.send({}, dest=0)
    print(rank, "entering the while loop")
    while True:
        tile = comm.recv(source = 0)
        try:
            tile_results = {}
            for tuple_ in tuple_generator(tile):
                
                #IGs = slow_work(tuple_)
                IGs = fast.work_3a(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
                #IGs = fast.work_3b(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
                #IGs = fast.work_3c(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
                #IGs = fast.work_3_old(dim1, divisions, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])
                
                dof = np.prod(tuple(bucket_counts[col_idx] for col_idx in tuple_), dtype = 'int')
                record_tuple(tuple_, IGs, dof, tile_results)
            comm.isend(tile_results, dest=0)
        except:
            print(rank, "says goodbye")
            break


Overwriting march2_mpi.py


In [16]:
%%bash
export OMP_NUM_THREADS=1
mpirun -n 2 python march2_mpi.py

0 dim0 = 50 ; dim1 = 2000
1 entering the while loop
0 entering the for loop
0 Work queue is empty
1 says goodbye
0 says goodbye
0 Elapsed: 2.3193969349958934 sec


In [17]:
import pandas as pd
import pickle
with open("march2_mpi_results.pkl", "rb") as file:
    final_results_mpi = pickle.load(file)

for dof, dict_ in final_results_mpi.items():
    
    df = pd.DataFrame(dict_).T.rename(columns={0: 'IG_max', 1: 'tuple'})
    df.sort_values('IG_max', ascending=False, inplace=True)

    print("product of X's bucket-counts = {}".format(dof))
    display(df)
    print("---")

product of X's bucket-counts = 128


,IG_max,tuple
1,273.836,"[1, 43, 25]"
22,268.547,"[43, 22, 25]"
25,239.06,"[1, 43, 25]"
43,221.95,"[1, 43, 25]"
3,205.555,"[3, 22, 25]"
14,124.049,"[6, 14, 25]"
6,121.529,"[6, 14, 25]"
42,118.579,"[0, 42, 12]"
12,118.251,"[0, 42, 12]"
21,117.97,"[21, 25, 32]"


---
product of X's bucket-counts = 512


,IG_max,tuple
43,469.58,"[1, 43, 22]"
3,457.692,"[1, 3, 22]"
22,412.102,"[3, 41, 22]"
1,407.952,"[1, 43, 32]"
9,392.656,"[7, 9, 14]"
18,390.184,"[47, 18, 37]"
7,383.371,"[7, 9, 14]"
14,379.428,"[7, 9, 14]"
38,373.26,"[6, 12, 38]"
48,372.887,"[48, 12, 17]"


---
product of X's bucket-counts = 64


...

---
product of X's bucket-counts = 192


,IG_max,tuple
43,284.9,"[1, 5, 43]"
22,278.071,"[43, 5, 22]"
3,265.697,"[1, 3, 5]"
1,257.315,"[1, 5, 43]"
5,180.757,"[43, 5, 22]"
21,166.651,"[5, 21, 27]"
26,166.587,"[5, 13, 26]"
28,165.338,"[17, 28, 30]"
17,163.5,"[17, 28, 30]"
41,162.253,"[41, 22, 30]"


---
product of X's bucket-counts = 72


,IG_max,tuple
43,103.065,"[43, 5, 30]"
22,101.528,"[5, 22, 30]"
25,96.6994,"[45, 20, 25]"
3,88.8426,"[3, 5, 30]"
5,68.595,"[5, 22, 30]"
21,61.0716,"[5, 21, 30]"
9,60.6375,"[5, 9, 30]"
38,59.7251,"[5, 30, 38]"
41,56.4259,"[41, 5, 30]"
46,55.5272,"[5, 30, 46]"


---


Compare (manually) the results obtained with and without `mpi`

In [18]:
for key_mpi, dict_mpi in final_results_mpi.items():
    print(key_mpi)
    for column, result in dict_mpi.items():
        print("mpi: ", column, result)
        print(" n1: ", column, final_results_n1[key_mpi][column])
        print("--")
    print("--")

128
mpi:  0 (82.2360588224874, [0, 2, 22])
 n1:  0 (82.23605882248421, (0, 2, 22))
--
mpi:  1 (273.8360245123258, [1, 43, 25])
 n1:  1 (273.83602451232355, (1, 25, 43))
--
mpi:  2 (106.37081912636222, [2, 13, 25])
 n1:  2 (106.37081912635131, (2, 13, 25))
--
mpi:  3 (205.55459762079136, [3, 22, 25])
 n1:  3 (205.55459762078863, (3, 22, 25))
--
mpi:  4 (113.47304122763398, [4, 25, 33])
 n1:  4 (113.47304122762307, (4, 25, 33))
--
mpi:  6 (121.52879498170296, [6, 14, 25])
 n1:  6 (121.52879498169705, (6, 14, 25))
--
mpi:  7 (105.29950717678275, [4, 7, 25])
 n1:  7 (105.29950717677457, (4, 7, 25))
--
mpi:  8 (111.97358856250776, [0, 8, 21])
 n1:  8 (111.97358856250776, (0, 8, 21))
--
mpi:  9 (110.66435617404386, [0, 9, 39])
 n1:  9 (110.66435617404386, (0, 9, 39))
--
mpi:  10 (95.26860257119824, [10, 11, 35])
 n1:  10 (95.26860257119779, (10, 11, 35))
--
mpi:  11 (107.10007056155246, [10, 11, 35])
 n1:  11 (107.10007056154973, (10, 11, 35))
--
mpi:  12 (118.25052955910269, [0, 42, 12])
 n

 n1:  30 (23.012063495372786, (5, 10, 30))
--
--
90
mpi:  20 (91.50530712091586, [15, 20, 30])
 n1:  20 (91.50530712091495, (15, 20, 30))
--
mpi:  5 (52.32810104516648, [40, 5, 45])
 n1:  5 (52.32810104516193, (5, 40, 45))
--
mpi:  40 (64.04020651221799, [40, 5, 45])
 n1:  40 (64.04020651221344, (5, 40, 45))
--
mpi:  45 (66.07779360529094, [40, 5, 45])
 n1:  45 (66.07779360528912, (5, 40, 45))
--
mpi:  30 (82.71592647941088, [15, 20, 30])
 n1:  30 (82.71592647941088, (15, 20, 30))
--
mpi:  15 (101.25374568125244, [15, 20, 30])
 n1:  15 (101.2537456812588, (15, 20, 30))
--
--
384
mpi:  1 (340.83608086499953, [1, 43, 45])
 n1:  1 (340.8360808649959, (1, 43, 45))
--
mpi:  2 (287.1714748033364, [2, 7, 45])
 n1:  2 (287.1714748033412, (2, 7, 45))
--
mpi:  3 (342.48379488310684, [1, 3, 45])
 n1:  3 (342.4837948831098, (1, 3, 45))
--
mpi:  4 (284.5437082686508, [4, 45, 46])
 n1:  4 (284.5437082686433, (4, 45, 46))
--
mpi:  6 (285.71030915570145, [6, 20, 29])
 n1:  6 (285.71030915569804, (6, 2

mpi:  31 (163.79917197771374, [40, 45, 31])
 n1:  31 (163.79917197769646, (31, 40, 45))
--
mpi:  32 (172.57846361687734, [45, 15, 32])
 n1:  32 (172.5784636168587, (15, 32, 45))
--
mpi:  33 (164.30315324624007, [40, 45, 33])
 n1:  33 (164.30315324622188, (33, 40, 45))
--
mpi:  34 (186.22205768402182, [15, 20, 34])
 n1:  34 (186.22205768401045, (15, 20, 34))
--
mpi:  36 (183.4285685581608, [15, 20, 36])
 n1:  36 (183.42856855815262, (15, 20, 36))
--
mpi:  37 (172.24537358570615, [15, 20, 37])
 n1:  37 (172.24537358569796, (15, 20, 37))
--
mpi:  38 (170.02806125764164, [45, 15, 38])
 n1:  38 (170.02806125763118, (15, 38, 45))
--
mpi:  39 (149.9781631384194, [40, 45, 39])
 n1:  39 (149.9781631383953, (39, 40, 45))
--
mpi:  40 (177.71077779934058, [40, 45, 48])
 n1:  40 (177.71077779933375, (40, 45, 48))
--
mpi:  41 (184.694675354609, [41, 15, 20])
 n1:  41 (184.694675354604, (15, 20, 41))
--
mpi:  42 (156.45936795014768, [40, 42, 45])
 n1:  42 (156.4593679501295, (40, 42, 45))
--
mpi:  43